In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$



Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавая свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками.

Класс MeanTargetEncoderNoise должен иметь следующую сигнатуру:



In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric, C=0.006):              
        ### Your code is here
        self.categorical = categorical
        self.numeric = numeric
        self.C = C
    
    def fit(self, X, y):
        ### Your code is here
        X_fit = X.copy()
        y_fit = y.copy()
        self.dict_of_means = {col: y_fit.groupby(X_fit[col]).mean() + 
                              self.C * np.random.normal(size=len(y_fit.groupby(X_fit[col]))) 
                              for col in self.categorical}
        return self
        
    def transform(self, df):
        ### Your code is here
        temp = df.copy()
        for col in self.categorical:
            temp[col] = temp[col].map(self.dict_of_means[col]).fillna(0)  
        return temp

Разделите колонки на вещественные и категориальные. Приведите все категориальные колонки к типу `object`.

Далее применим наш кодировщик к `X_train, X_test`, так же как например мы применяем `StandardScaler`, чтобы проверить работоспособность нашего класса. Установите зерно датчика случайный чисел `np.random.seed(1)`.

После того, как вы изменили обучающую и тестовую выборки, сохраните первые 10 строк полученного промежуточного датафрейма обучающей выборки (`X_train`) в файл в формате csv с сепаратором `;`. Не забудьте индекс. Отправьте полученный файл в форму ниже.

Список колонок которые должны быть в файле для сдачи:
```py
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]
```

### Ваше решение


Разделение колонок на категориальные и числовые.

In [6]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


Проверка работы трансформера.

In [7]:
np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head(10).to_csv('train.csv', sep=';', index=False)
train.head(10)

,name,year,km_driven,fuel,seller_type,transmission,owner
3294,13.483692,13.436559,50000,13.093756,12.615863,13.771135,12.977730
2290,12.117029,11.903115,70000,12.453832,12.615863,13.771135,12.977730
874,12.310456,13.328864,50000,12.453832,12.615863,12.639805,12.977730
1907,12.491443,13.042359,92198,12.453832,13.152824,12.639805,12.463313
3244,12.390906,12.870886,3240,12.453832,12.615863,12.639805,12.463313
1089,12.687432,13.436559,10000,12.453832,13.152824,12.639805,12.977730
3902,11.698702,11.503504,90000,12.453832,12.615863,12.639805,11.878390
2215,11.120678,11.503504,79000,12.453832,12.615863,12.639805,12.463313
3862,13.172879,13.328864,99700,13.093756,12.615863,12.639805,12.977730
705,13.004194,12.241213,124000,13.093756,12.615863,12.639805,12.463313


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [8]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [9]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here
max_dpth = {}
min_samles_split = {}
min_impurity_decrease = {}
max_leaf_nodes = {}
for max_depth in max_depth_list:
    pipe = Pipeline([('transformer', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                     ('tree', DecisionTreeRegressor(max_depth=max_depth))])
    pipe.fit(X_train, y_train)
    max_dpth[max_depth] = mse(y_test, pipe.predict(X_test)).round(2)
for min_sample in min_samples_split_list:
    pipe = Pipeline([('transformer', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                     ('tree', DecisionTreeRegressor(min_samples_split=min_sample))])
    pipe.fit(X_train, y_train)
    min_samles_split[min_sample] = mse(y_test, pipe.predict(X_test)).round(2)
for min_impurity in min_impurity_decrease_list:
    pipe = Pipeline([('transformer', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                     ('tree', DecisionTreeRegressor(min_impurity_decrease=min_impurity))])
    pipe.fit(X_train, y_train)
    min_impurity_decrease[min_impurity] = mse(y_test, pipe.predict(X_test)).round(2)
for max_leaf in max_leaf_nodes_list:
    pipe = Pipeline([('transformer', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                     ('tree', DecisionTreeRegressor(max_leaf_nodes=max_leaf))])
    pipe.fit(X_train, y_train)
    max_leaf_nodes[max_leaf] = mse(y_test, pipe.predict(X_test)).round(2)
print("MSE for max_depth_list", max_dpth)
print("MSE for min_samples_split_list", min_samles_split)
print("MSE for min_impurity_decrease_list", min_impurity_decrease)
print("MSE for max_leaf_nodes_list", max_leaf_nodes)

MSE for max_depth_list {3: 0.8, 5: 1.44, 8: 1.99, 12: 1.98}
MSE for min_samples_split_list {10: 1.43, 50: 1.44, 100: 0.95, 500: 0.81}
MSE for min_impurity_decrease_list {0: 1.99, 0.1: 0.52, 0.15: 0.52, 0.2: 0.52}
MSE for max_leaf_nodes_list {100: 1.99, 200: 1.99, 500: 1.99}


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "tree__max_depth": [3, 5, 8, 12],
    "tree__min_samples_split": [10, 50, 100, 500],
    "tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here
pipe = Pipeline([('transformer', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
                 ('tree', DecisionTreeRegressor())])
pipe.fit(X_train, y_train)
spliter = [(X_train.index.to_list(), X_test.index.to_list())]
search = GridSearchCV(pipe, param_grid, cv=4, scoring='neg_mean_squared_error')
search.fit(X_train, y_train)
print("Best params:", search.best_params_)
print("Best score:", round(np.mean((y_test - search.best_estimator_.predict(X_test)) ** 2), 2))

Best params: {'tree__max_depth': 12, 'tree__max_leaf_nodes': 100, 'tree__min_impurity_decrease': 0.15, 'tree__min_samples_split': 100}
Best score: 0.52
